In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
!pip install pytorch-transformers
!pip install transformers
!pip install nltk


     |████████████████████████████████| 184kB 18.7MB/s 
     |████████████████████████████████| 890kB 43.5MB/s 
     |████████████████████████████████| 1.2MB 52.3MB/s 
     |████████████████████████████████| 133kB 56.6MB/s 
     |████████████████████████████████| 7.3MB 53.3MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=55475bf39ba597a89fe7e01e163d8c33802c3163214bd07b43e4b57851f53753
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.20.28 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.9MB 18.3MB/s 
     |████████████████████████████████| 3.2MB 51.5MB/s 


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
!pip install tweet-preprocessor

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import DebertaTokenizer, DebertaModel, DebertaForSequenceClassification
from transformers import AdamW
import nltk
from nltk.stem import 	WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# % matplotlib inline


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [7]:
MAX_LEN = 128

In [8]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/FakeNewsNet/fnn_train.csv')


In [9]:
train = train.rename(columns={ 'fullText_based_content' : 'text'})
train = train.rename(columns={ 'label_fnn' : 'label'})
train.loc[train['label'] == 'fake', 'label'] = '0'
train.loc[train['label'] == 'real', 'label'] = '1'

In [10]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 9127
Size of validation set: 3042
Size of testing set: 3043


In [11]:
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()


In [12]:
p.set_options(p.OPT.URL, p.OPT.EMOJI)

def preprocess(row, lemmatizer, stemmer):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  tokenization = nltk.word_tokenize(text)     
  tokenization = [w for w in tokenization if not w in stop_words]
#   text = ' '.join([porter_stemmer.stem(w) for w in tokenization])
#   text = ' '.join([lemmatizer.lemmatize(w) for w in tokenization])
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [13]:
df['text'] = df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
val_df['text'] = val_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
test_df['text'] = test_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)


In [14]:
def map_label(row):
  return 0 if row['label']=='1' else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [15]:
train_sentences = df.text.values
val_sentences = val_df.text.values
test_sentences = test_df.text.values

train_labels = df.label_encoded.values
val_labels = val_df.label_encoded.values


In [ ]:
# tokenizer_sentences = np.array(list(df.tweet.values) + list(val_df.tweet.values))

In [16]:
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base', do_lower_case=True)


In [17]:
def Encode_TextWithAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=True):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids'],encoded_dict['attention_mask']

def Encode_TextWithoutAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=False):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids']

def get_TokenizedTextWithAttentionMask(sentenceList, tokenizer):
    token_ids_list,attention_mask_list = [],[]
    for sentence in sentenceList:
        token_ids,attention_mask = Encode_TextWithAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
        attention_mask_list.append(attention_mask)
    return token_ids_list,attention_mask_list

def get_TokenizedText(sentenceList, tokenizer):
    token_ids_list = []
    for sentence in sentenceList:
        token_ids = Encode_TextWithoutAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
    return token_ids_list

In [18]:
train_token_ids,train_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(train_sentences,tokenizer))
val_token_ids,val_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(val_sentences,tokenizer))
test_token_ids,test_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(test_sentences,tokenizer))

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

In [19]:
batch_size = 32

train_data = TensorDataset(train_token_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(val_token_ids, val_attention_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_token_ids, test_attention_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [20]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2).cuda()

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base

In [21]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [22]:
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [23]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
train_loss_set = []
best_val_accuracy = 0.50
directory_path =  '/content/gdrive/MyDrive/4th_sem/dataset/FakeNewsNet'
epochs = 10

for _ in trange(epochs, desc="Epoch"):
  
  model.train()
  
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    optimizer.zero_grad()
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    loss.backward()
    optimizer.step()
        
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
  model.eval()

  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  Validation_Accuracy = (eval_accuracy/nb_eval_steps)
  if(Validation_Accuracy >= best_val_accuracy):
    torch.save(model.state_dict(), directory_path+'/models/DeBERTa_base_best_model.ckpt')
    best_val_accuracy = Validation_Accuracy
    print('Model Saved')

    


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.6666064318660256
Validation Accuracy: 0.6276041666666666


Epoch:  10%|█         | 1/10 [04:55<44:23, 295.94s/it]

Model Saved
Train loss: 0.6166979719500442


Epoch:  20%|██        | 2/10 [09:56<39:39, 297.46s/it]

...

Validation Accuracy: 0.4820963541666667
Train loss: 0.6938514411449432


Epoch:  90%|█████████ | 9/10 [44:53<04:59, 299.38s/it]

Validation Accuracy: 0.4820963541666667
Train loss: 0.6938346870295651


Epoch: 100%|██████████| 10/10 [49:53<00:00, 299.30s/it]

Validation Accuracy: 0.4820963541666667


In [25]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import XLNetTokenizer, XLNetForSequenceClassification,BertTokenizer, BertForSequenceClassification, DebertaTokenizer, DebertaForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification, ElectraTokenizer, ElectraForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [27]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/FakeNewsNet/fnn_train.csv')

In [28]:
train = train.rename(columns={ 'fullText_based_content' : 'text'})
train = train.rename(columns={ 'label_fnn' : 'label'})
train.loc[train['label'] == 'fake', 'label'] = '0'
train.loc[train['label'] == 'real', 'label'] = '1'

In [29]:
train.head()

,id,date,speaker,statement,sources,paragraph_based_content,text,label
0,3106,2011-01-25T06:00:00-05:00,Joe Wilkinson,A national organization says Georgia has one o...,['http://www.ajc.com/news/georgia-politics-ele...,['A coalition of government watchdog groups la...,A coalition of government watchdog groups last...,0
1,5655,2012-04-02T11:42:20-04:00,Rick Scott,"Says Barack Obama's health care law ""will be t...",['http://www.youtube.com/watch?v=TaC0mKApf9Q&f...,['As Supreme Court justices embarked on three ...,As Supreme Court justices embarked on three da...,0
2,3506,2011-04-01T09:49:05-04:00,J.D. Alexander,Says the Southwest Florida Water Management Di...,['http://www.tampabay.com/news/politics/gubern...,"[""Here's a new one: The Senate budget committe...",Here's a new one: The Senate budget committee ...,0
3,3450,2011-03-21T12:20:02-04:00,Paul Ryan,"""The Congressional Budget Office has this econ...",['http://www.cnn.com/2011/POLITICS/03/17/gop.b...,"['Recently, House Budget chairman Paul Ryan, R...","Recently, House Budget chairman Paul Ryan, R-W...",1
4,4776,2011-11-13T07:30:00-05:00,Rodney Frelinghuysen,"Says the Treasury Department ""says 41 percent ...",['http://frelinghuysen.house.gov/index.cfm?sec...,['The millionaires’ tax proposal made its late...,The millionaires’ tax proposal made its latest...,0


In [30]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
test_label_df = test_df

print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 9127
Size of validation set: 3042
Size of testing set: 3043


In [32]:
def preprocess(row):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [33]:
def map_label(row):
  return 0 if row['label']=='1' else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [34]:
train_sentences = df.text.values
train_token_ids = df.id.values
val_sentences = val_df.text.values
val_token_ids = val_df.id.values
test_sentences = test_df.text.values
test_token_ids = test_df.id.values


In [35]:
train_sentences = [sentence + " [SEP] [CLS]" for sentence in train_sentences]
train_labels = df.label_encoded.values
val_sentences = [sentence + " [SEP] [CLS]" for sentence in val_sentences]
val_labels = val_df.label_encoded.values
test_sentences = [sentence + " [SEP] [CLS]" for sentence in test_sentences]


In [36]:
MAX_LEN = 128
batch_size = 64


In [37]:
def get_dataloader(network, train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids):
  dic = {"XLNET": "xlnet-base-cased", "BERT": "bert-base-uncased", "ROBERT": "roberta-base", "XLM-ROBERT": "xlm-roberta-base", "DeBERTa": "microsoft/deberta-base", "Electra": "google/electra-base-discriminator"}
  if network=='XLNET':
    tokenizer = XLNetTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='DeBERTa':
    tokenizer = DebertaTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='BERT':
    tokenizer = BertTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='Electra':
    tokenizer = ElectraTokenizer.from_pretrained(dic[network], do_lower_case=True)
  else:
    tokenizer = RobertaTokenizerFast.from_pretrained(dic[network], do_lower_case=True)

  tokenized_train_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
  print ("Tokenize the first sentence:")
  print (tokenized_train_texts[0])

  tokenized_val_texts = [tokenizer.tokenize(sent) for sent in val_sentences]
  tokenized_test_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
  input_train_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_train_texts]
  input_val_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_val_texts]
  input_test_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_test_texts]

  input_train_ids = pad_sequences(input_train_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_val_ids = pad_sequences(input_val_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_test_ids = pad_sequences(input_test_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Create attention masks
  train_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_train_ids:
    seq_mask = [float(i>0) for i in seq]
    train_attention_masks.append(seq_mask)
  train_masks = np.array(train_attention_masks)

  # Create attention masks
  val_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_val_ids:
    seq_mask = [float(i>0) for i in seq]
    val_attention_masks.append(seq_mask)
  validation_masks = np.array(val_attention_masks)

  # Create attention masks
  test_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_test_ids:
    seq_mask = [float(i>0) for i in seq]
    test_attention_masks.append(seq_mask)
  test_masks = np.array(test_attention_masks)

  train_inputs = torch.tensor(input_train_ids)
  validation_inputs = torch.tensor(input_val_ids)
  test_inputs = torch.tensor(input_test_ids)
  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(val_labels)
  train_masks = torch.tensor(train_masks)
  validation_masks = torch.tensor(validation_masks)
  test_masks = torch.tensor(test_masks)

  train_data = TensorDataset(torch.tensor(train_token_ids), train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  validation_data = TensorDataset(torch.tensor(val_token_ids), validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  test_data = TensorDataset(torch.tensor(test_token_ids), test_inputs, test_masks)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  return train_dataloader, validation_dataloader, test_dataloader


In [38]:
train_dataloader_DeBERTa, val_dataloader_DeBERTa, test_dataloader_DeBERTa = get_dataloader("DeBERTa", train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids)

Tokenize the first sentence:
['321', '312', '257', '3957', '9359', '326', '1714', '14616', '318', '257', '1917', '287', '9529', '1098', '7022', '11', '262', '1171', '1895', '1430', '366', '1181', '286', '262', '1181', '366', '7025', '257', '5114', '286', '262', '2071', '764', '1141', '262', '905', '11', '2583', '302', '20627', '627', '328', '1636', '4438', '1811', '7869', '546', '1714', '3259', '13', '530', '287', '1948', '4978', '674', '1027', '13', '198', '1', '27078', '257', '2415', '14170', '656', '262', '1714', '3292', '553', '673', '531', '11', '366', '372', '2811', '1204', '29098', '318', '3598', '812', '11', '351', '31378', '290', '19625', '355', '262', '1353', '25542', '526', '198', '568', '611', '257', '1248', '12', '1941', '12', '727', '2415', '4940', '1762', '355', '257', '37833', '11', '673', '338', '1884', '284', '307', '2636', '416', '2479', '1679', '30', '198', '732', '3066', '284', '2198', '503', '326', '28695', '24696', '13', '198', '732', '1043', '262', '1624', '2221

In [39]:
model4 = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)
directory_path = '/content/gdrive/MyDrive/4th_sem/dataset/FakeNewsNet/models'
model4.load_state_dict(torch.load(directory_path+'/DeBERTa_base_best_model.ckpt'))
model4.eval()
model4.cuda()



Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

In [40]:
def get_model_preds_labels(model, dataloader, mode='train'):
  data_vectors = []
  labels = []
  ids = []

  with torch.no_grad():
      correct = 0
      total = 0
      for i, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        if mode=='test':
            token_ids, b_input_ids, b_input_mask = batch
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            for point, tk_id in zip(F.softmax(outputs.logits).tolist(), token_ids.tolist()):
              data_vectors.append(point)       
              ids.append(tk_id)     
        else:
          # Unpack the inputs from our dataloader
          token_ids, b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          for point, lab, tk_id in zip(F.softmax(outputs.logits).tolist(), b_labels.tolist(), token_ids.tolist()):
            data_vectors.append(point)
            labels.append(lab)
            ids.append(tk_id)

  if mode=='test':
    return data_vectors, ids
  else:
    return data_vectors, labels, ids


#Train Data

In [41]:
m4, _, train_ids4 = get_model_preds_labels(model4, train_dataloader_DeBERTa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [42]:
pred_labels = ['1' if np.argmax(np.array([each]), 1)[0]==0 else '0' for each in m4]
train_pred_df = pd.DataFrame({'id': train_ids4, 'predicted_label': pred_labels})
train_pred_df = df[["id", "label"]].merge(train_pred_df, on='id', how='left')
train_pred_df.head()

,id,label,predicted_label
0,10883,0,1
1,10444,1,1
2,13513,0,1
3,8777,1,0
4,1514,0,0


In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score


In [44]:
confusion_matrix(train_pred_df['label'].values, train_pred_df['predicted_label'].values)

array([[3043, 1503],
       [1820, 2761]])

In [45]:
print(accuracy_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values))
print(precision_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(recall_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(f1_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))


0.6359154157992769
0.6359154157992769
0.6359154157992769
0.6359154157992769


#Validation Data

In [46]:
v4, _, v_ids4 = get_model_preds_labels(model4, val_dataloader_DeBERTa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [47]:
val_pred_labels = ['1' if np.argmax(np.array([each]), 1)[0]==0 else '0' for each in v4]
val_pred_df = pd.DataFrame({'id': v_ids4, 'predicted_label': val_pred_labels})
val_pred_df = val_df[["id", "label"]].merge(val_pred_df, on='id', how='left')
val_pred_df.head()



,id,label,predicted_label
0,6264,0,0
1,2851,0,0
2,9316,1,0
3,6004,1,1
4,9149,0,0


In [48]:
confusion_matrix(val_pred_df['label'].values, val_pred_df['predicted_label'].values)

array([[1018,  484],
       [ 632,  908]])

In [49]:
print(accuracy_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values))
print(precision_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(recall_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(f1_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))

0.6331360946745562
0.6331360946745562
0.6331360946745562
0.6331360946745562


#Test Data

In [50]:
t4, t_ids4 = get_model_preds_labels(model4, test_dataloader_DeBERTa, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [51]:
test_pred_labels = ['1' if np.argmax(np.array([each]), 1)[0]==0 else '0' for each in t4]
test_pred_df = pd.DataFrame({'id': t_ids4, 'predicted_label': test_pred_labels})
test_pred_df = test_label_df[["id", "label"]].merge(test_pred_df, on='id', how='left')
test_pred_df.head()

,id,label,predicted_label
0,2912,1,1
1,2666,0,0
2,4851,0,0
3,7565,1,0
4,6438,0,0


In [52]:
confusion_matrix(test_pred_df['label'].values, test_pred_df['predicted_label'].values)

array([[1065,  508],
       [ 611,  859]])

In [53]:
print(accuracy_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values))
print(precision_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))
print(recall_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))
print(f1_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))

0.6322707854091357
0.6322707854091357
0.6322707854091357
0.6322707854091357
